In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import datetime as dt
import calendar,warnings,itertools,matplotlib,keras,shutil
import tensorflow as tf
import statsmodels.api as sm
from datetime import datetime
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression,ElasticNet, Lasso
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingRegressor,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc,roc_auc_score,classification_report
from sklearn.datasets import load_iris,make_regression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.kernel_ridge import KernelRidge
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from IPython.core import display as ICD
# from tensorflow_core.estimator import inputs
from imblearn.over_sampling import SMOTE
#Hiding the warnings
warnings.filterwarnings('ignore')


In [2]:
#Importing Dataset using pandas
dataset=pd.read_csv("/content/DataCoSupplyChainDataset.csv",header= 0, encoding="ISO-8859-1")
dataset.head(5)

,ï»¿Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/18 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/18 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/18 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/18 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/18 11:24,Standard Class


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ï»¿Type                        180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [4]:
dataset.apply(lambda x: sum(x.isnull())) #Checking missing values

,0
ï»¿Type,0
Days for shipping (real),0
Days for shipment (scheduled),0
Benefit per order,0
Sales per customer,0
Delivery Status,0
Late_delivery_risk,0
Category Id,0
Category Name,0
Customer City,0


In [5]:
dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str)+dataset['Customer Lname'].astype(str)

In [6]:
data=dataset.drop(['Customer Email','Product Status','Customer Password','Customer Street','Customer Fname','Customer Lname',
           'Latitude','Longitude','Product Description','Product Image','Order Zipcode','shipping date (DateOrders)'],axis=1)
data.shape

(180519, 42)

In [7]:
data['Customer Zipcode']=data['Customer Zipcode'].fillna(0)

In [8]:
data.head()

,ï»¿Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Profit Per Order,Order Region,Order State,Order Status,Product Card Id,Product Category Id,Product Name,Product Price,Shipping Mode,Customer Full Name
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,91.250000,Southeast Asia,Java Occidental,COMPLETE,1360,73,Smart watch,327.75,Standard Class,CallyHolloway
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,-249.089996,South Asia,RajastÂ·n,PENDING,1360,73,Smart watch,327.75,Standard Class,IreneLuna
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,-247.779999,South Asia,RajastÂ·n,CLOSED,1360,73,Smart watch,327.75,Standard Class,GillianMaldonado
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,22.860001,Oceania,Queensland,COMPLETE,1360,73,Smart watch,327.75,Standard Class,TanaTate
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,134.210007,Oceania,Queensland,PENDING_PAYMENT,1360,73,Smart watch,327.75,Standard Class,OrliHendricks


In [9]:
data['Order Status'].value_counts()

,count
Order Status,
COMPLETE,59491
PENDING_PAYMENT,39832
PROCESSING,21902
PENDING,20227
CLOSED,19616
ON_HOLD,9804
SUSPECTED_FRAUD,4062
CANCELED,3692
PAYMENT_REVIEW,1893


In [10]:
data['fraud'] = np.where(data['Order Status'] == 'SUSPECTED_FRAUD', 1, 0)

In [11]:
data["Delivery Status"].value_counts()

,count
Delivery Status,
Late delivery,98977
Advance shipping,41592
Shipping on time,32196
Shipping canceled,7754


In [12]:
data['late_delivery']=np.where(data['Delivery Status'] == 'Late delivery', 1, 0)

In [13]:
data["fraud"].value_counts()

,count
fraud,
0,176457
1,4062


In [14]:
data.drop(['Delivery Status','Late_delivery_risk','Order Status','order date (DateOrders)'], axis=1, inplace=True)

In [15]:
data.head()

,ï»¿Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Category Id,Category Name,Customer City,Customer Country,Customer Id,...,Order Region,Order State,Product Card Id,Product Category Id,Product Name,Product Price,Shipping Mode,Customer Full Name,fraud,late_delivery
0,DEBIT,3,4,91.250000,314.640015,73,Sporting Goods,Caguas,Puerto Rico,20755,...,Southeast Asia,Java Occidental,1360,73,Smart watch,327.75,Standard Class,CallyHolloway,0,0
1,TRANSFER,5,4,-249.089996,311.359985,73,Sporting Goods,Caguas,Puerto Rico,19492,...,South Asia,RajastÂ·n,1360,73,Smart watch,327.75,Standard Class,IreneLuna,0,1
2,CASH,4,4,-247.779999,309.720001,73,Sporting Goods,San Jose,EE. UU.,19491,...,South Asia,RajastÂ·n,1360,73,Smart watch,327.75,Standard Class,GillianMaldonado,0,0
3,DEBIT,3,4,22.860001,304.809998,73,Sporting Goods,Los Angeles,EE. UU.,19490,...,Oceania,Queensland,1360,73,Smart watch,327.75,Standard Class,TanaTate,0,0
4,PAYMENT,2,4,134.210007,298.250000,73,Sporting Goods,Caguas,Puerto Rico,19489,...,Oceania,Queensland,1360,73,Smart watch,327.75,Standard Class,OrliHendricks,0,0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ï»¿Type                        180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Category Id                    180519 non-null  int64  
 6   Category Name                  180519 non-null  object 
 7   Customer City                  180519 non-null  object 
 8   Customer Country               180519 non-null  object 
 9   Customer Id                    180519 non-null  int64  
 10  Customer Segment               180519 non-null  object 
 11  Customer State                 180519 non-null  object 
 12  Customer Zipcode              

In [19]:
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
data['Customer Country']  = le.fit_transform(data['Customer Country'])
data['Market']            = le.fit_transform(data['Market'])
data['ï»¿Type']              = le.fit_transform(data['ï»¿Type'])
data['Product Name']      = le.fit_transform(data['Product Name'])
data['Customer Segment']  = le.fit_transform(data['Customer Segment'])
data['Customer State']    = le.fit_transform(data['Customer State'])
data['Order Region']      = le.fit_transform(data['Order Region'])
data['Order City']        = le.fit_transform(data['Order City'])
data['Category Name']     = le.fit_transform(data['Category Name'])
data['Customer City']     = le.fit_transform(data['Customer City'])
data['Department Name']   = le.fit_transform(data['Department Name'])
data['Order State']       = le.fit_transform(data['Order State'])
data['Shipping Mode']     = le.fit_transform(data['Shipping Mode'])
data['Order Country']     = le.fit_transform(data['Order Country'])
data['Customer Full Name']= le.fit_transform(data['Customer Full Name'])

#display the initial records
data.head()

,ï»¿Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Category Id,Category Name,Customer City,Customer Country,Customer Id,...,Order Region,Order State,Product Card Id,Product Category Id,Product Name,Product Price,Shipping Mode,Customer Full Name,fraud,late_delivery
0,1,3,4,91.250000,314.640015,73,40,66,1,20755,...,15,475,1360,73,78,327.75,3,1876,0,0
1,3,5,4,-249.089996,311.359985,73,40,66,1,19492,...,13,841,1360,73,78,327.75,3,5378,0,1
2,0,4,4,-247.779999,309.720001,73,40,452,0,19491,...,13,841,1360,73,78,327.75,3,4429,0,0
3,1,3,4,22.860001,304.809998,73,40,285,0,19490,...,11,835,1360,73,78,327.75,3,12929,0,0
4,2,2,4,134.210007,298.250000,73,40,66,1,19489,...,11,835,1360,73,78,327.75,3,10638,0,0


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ï»¿Type                        180519 non-null  int64  
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Category Id                    180519 non-null  int64  
 6   Category Name                  180519 non-null  int64  
 7   Customer City                  180519 non-null  int64  
 8   Customer Country               180519 non-null  int64  
 9   Customer Id                    180519 non-null  int64  
 10  Customer Segment               180519 non-null  int64  
 11  Customer State                 180519 non-null  int64  
 12  Customer Zipcode              

In [21]:
data["fraud"].value_counts()

,count
fraud,
0,176457
1,4062


In [22]:
data.to_csv('fraud.csv', index=False)

In [ ]:

# 1. Split into features/target
X = data.drop(columns=['fraud'])
y = data['fraud']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [ ]:
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train)

In [ ]:
print(pd.Series(y_train_res).value_counts())

fraud
0    141165
1    141165
Name: count, dtype: int64


In [ ]:
from sklearn.ensemble            import (RandomForestClassifier, ExtraTreesClassifier,
                                         AdaBoostClassifier, GradientBoostingClassifier)
from sklearn.linear_model        import (LogisticRegression, SGDClassifier, RidgeClassifier,
                                         PassiveAggressiveClassifier)
from sklearn.svm                 import SVC
from sklearn.neighbors           import KNeighborsClassifier
from sklearn.tree                import DecisionTreeClassifier
from sklearn.naive_bayes         import (GaussianNB, BernoulliNB, ComplementNB)
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA,
                                           QuadraticDiscriminantAnalysis as QDA)
from sklearn.neural_network      import MLPClassifier
from xgboost                     import XGBClassifier
from lightgbm                    import LGBMClassifier
from catboost                    import CatBoostClassifier


In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00


In [ ]:
classifiers = {
    "LogisticRegression":  LogisticRegression(max_iter=1000, n_jobs=-1, random_state=42),
    "RandomForest":        RandomForestClassifier(n_jobs=-1, random_state=42),
    "AdaBoost":            AdaBoostClassifier(random_state=42),
    "GradientBoost":       GradientBoostingClassifier(random_state=42),
    "XGBoost":             XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "LightGBM":            LGBMClassifier(random_state=42),
    "CatBoost":            CatBoostClassifier(verbose=0, random_state=42),
    "SVC":                 SVC(probability=True, random_state=42),
    "KNeighbors":          KNeighborsClassifier(),
    "DecisionTree":        DecisionTreeClassifier(random_state=42),
    "GaussianNB":          GaussianNB(),
    "BernoulliNB":         BernoulliNB(),
    "MLP":                 MLPClassifier(max_iter=500, random_state=42),
    "SGD":                 SGDClassifier(max_iter=1000, random_state=42),
    "PassiveAggressive":   PassiveAggressiveClassifier(max_iter=1000, random_state=42),
    "RidgeClassifier":     RidgeClassifier()
}

In [ ]:
for name, clf in classifiers.items():
    clf.fit(X_train_res, y_train_res)
    y_pred  = clf.predict(X_test_scaled)
    y_proba = clf.predict_proba(X_test_scaled)[:,1] if hasattr(clf, "predict_proba") else None

    print(f"{name}")
    print(f"  • Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    if y_proba is not None:
        print(f"  • ROC-AUC:  {roc_auc_score(y_test, y_proba):.4f}")
    print("  • Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    print("  • Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("="*80)

LogisticRegression
  • Accuracy: 0.8811
  • ROC-AUC:  0.9691
  • Classification Report:
              precision    recall  f1-score   support

           0     1.0000    0.8783    0.9352     35292
           1     0.1590    1.0000    0.2744       812

    accuracy                         0.8811     36104
   macro avg     0.5795    0.9392    0.6048     36104
weighted avg     0.9811    0.8811    0.9204     36104

  • Confusion Matrix:
[[30998  4294]
 [    0   812]]
RandomForest
  • Accuracy: 0.9849
  • ROC-AUC:  0.9873
  • Classification Report:
              precision    recall  f1-score   support

           0     0.9898    0.9948    0.9923     35292
           1     0.7102    0.5554    0.6234       812

    accuracy                         0.9849     36104
   macro avg     0.8500    0.7751    0.8078     36104
weighted avg     0.9835    0.9849    0.9840     36104

  • Confusion Matrix:
[[35108   184]
 [  361   451]]
AdaBoost
  • Accuracy: 0.8808
  • ROC-AUC:  0.9671
  • Classification 

In [ ]:
classifiers = {
    "CatBoost":            CatBoostClassifier(verbose=0, random_state=42),
    "SVC":                 SVC(probability=True, random_state=42),
    "KNeighbors":          KNeighborsClassifier(),
    "DecisionTree":        DecisionTreeClassifier(random_state=42),
    "GaussianNB":          GaussianNB(),
    "BernoulliNB":         BernoulliNB(),
    "MLP":                 MLPClassifier(max_iter=500, random_state=42),
    "SGD":                 SGDClassifier(max_iter=1000, random_state=42),
    "PassiveAggressive":   PassiveAggressiveClassifier(max_iter=1000, random_state=42),
    "RidgeClassifier":     RidgeClassifier()
}

In [ ]:
for name, clf in classifiers.items():
    clf.fit(X_train_res, y_train_res)
    y_pred  = clf.predict(X_test_scaled)
    y_proba = clf.predict_proba(X_test_scaled)[:,1] if hasattr(clf, "predict_proba") else None

    print(f"{name}")
    print(f"  • Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    if y_proba is not None:
        print(f"  • ROC-AUC:  {roc_auc_score(y_test, y_proba):.4f}")
    print("  • Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    print("  • Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("="*80)

CatBoost
  • Accuracy: 0.9875
  • ROC-AUC:  0.9895
  • Classification Report:
              precision    recall  f1-score   support

           0     0.9910    0.9962    0.9936     35292
           1     0.7863    0.6071    0.6852       812

    accuracy                         0.9875     36104
   macro avg     0.8886    0.8017    0.8394     36104
weighted avg     0.9864    0.9875    0.9867     36104

  • Confusion Matrix:
[[35158   134]
 [  319   493]]


In [ ]:
classifiers = {
    "RandomForest":        RandomForestClassifier(n_jobs=-1, random_state=42),
    "XGBoost":             XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "LightGBM":            LGBMClassifier(random_state=42),
    "CatBoost":            CatBoostClassifier(verbose=0, random_state=42),
    "SVC":                 SVC(probability=True, random_state=42),
    "KNeighbors":          KNeighborsClassifier(),
}

In [ ]:
for name, clf in classifiers.items():
    clf.fit(X_train_res, y_train_res)
    y_pred  = clf.predict(X_test_scaled)
    y_proba = clf.predict_proba(X_test_scaled)[:,1] if hasattr(clf, "predict_proba") else None

    print(f"{name}")
    print(f"  • Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    if y_proba is not None:
        print(f"  • ROC-AUC:  {roc_auc_score(y_test, y_proba):.4f}")
    print("  • Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    print("  • Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("="*80)

RandomForest
  • Accuracy: 0.9849
  • ROC-AUC:  0.9873
  • Classification Report:
              precision    recall  f1-score   support

           0     0.9898    0.9948    0.9923     35292
           1     0.7102    0.5554    0.6234       812

    accuracy                         0.9849     36104
   macro avg     0.8500    0.7751    0.8078     36104
weighted avg     0.9835    0.9849    0.9840     36104

  • Confusion Matrix:
[[35108   184]
 [  361   451]]
XGBoost
  • Accuracy: 0.9837
  • ROC-AUC:  0.9830
  • Classification Report:
              precision    recall  f1-score   support

           0     0.9890    0.9944    0.9917     35292
           1     0.6801    0.5185    0.5884       812

    accuracy                         0.9837     36104
   macro avg     0.8346    0.7564    0.7900     36104
weighted avg     0.9820    0.9837    0.9826     36104

  • Confusion Matrix:
[[35094   198]
 [  391   421]]
[LightGBM] [Info] Number of positive: 141165, number of negative: 141165
[LightGB

In [ ]:
 classifiers_1 = {"RandomForest":RandomForestClassifier(n_jobs=-1, random_state=42),
 "CatBoost":CatBoostClassifier(task_type="GPU", devices='0', verbose=0, random_state=42)}

In [ ]:
for name, clf in classifiers_1.items():
    clf.fit(X_train_res, y_train_res)
    y_pred  = clf.predict(X_test_scaled)
    y_proba = clf.predict_proba(X_test_scaled)[:,1] if hasattr(clf, "predict_proba") else None

    print(f"{name}")
    print(f"  • Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    if y_proba is not None:
        print(f"  • ROC-AUC:  {roc_auc_score(y_test, y_proba):.4f}")
    print("  • Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    print("  • Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("="*80)

RandomForest
  • Accuracy: 0.9849
  • ROC-AUC:  0.9873
  • Classification Report:
              precision    recall  f1-score   support

           0     0.9898    0.9948    0.9923     35292
           1     0.7102    0.5554    0.6234       812

    accuracy                         0.9849     36104
   macro avg     0.8500    0.7751    0.8078     36104
weighted avg     0.9835    0.9849    0.9840     36104

  • Confusion Matrix:
[[35108   184]
 [  361   451]]
CatBoost
  • Accuracy: 0.9796
  • ROC-AUC:  0.9768
  • Classification Report:
              precision    recall  f1-score   support

           0     0.9890    0.9901    0.9896     35292
           1     0.5486    0.5209    0.5344       812

    accuracy                         0.9796     36104
   macro avg     0.7688    0.7555    0.7620     36104
weighted avg     0.9791    0.9796    0.9793     36104

  • Confusion Matrix:
[[34944   348]
 [  389   423]]


In [ ]:
# Build the ANN model
model = Sequential()
model.add(Dense(128, input_dim=X_train_res.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Use 1 for binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_res, y_train_res,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    verbose=1)


Epoch 1/50
7059/7059 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.9148 - loss: 0.2035 - val_accuracy: 0.9884 - val_loss: 0.1046
Epoch 2/50
5256/7059 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9452 - loss: 0.1436

In [ ]:
y_pred_prob = model.predict(X_test_scaled).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_prob))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


1129/1129 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
Accuracy: 0.953107688898737
ROC-AUC: 0.9750508460310574
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98     35292
           1       0.30      0.81      0.44       812

    accuracy                           0.95     36104
   macro avg       0.65      0.88      0.71     36104
weighted avg       0.98      0.95      0.96     36104

Confusion Matrix:
 [[33756  1536]
 [  157   655]]
